In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import queue
from collections import namedtuple
from random import randint


Event = namedtuple('Event', 'time proc action')

NUM_TAXIS = 3
DEPARTURE_INTERVAL = 5


class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)

        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break

            current_event = self.events.get()
            sim_time, proc_id, prev_action = current_event
            print('taxi:', proc_id, proc_id * '    ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(prev_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))


def taxi_process(ident, trips, start_time=0):
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')

    yield Event(time, ident, 'going home')


def compute_duration(action):
    return randint(1, 10)


taxis = {
    i: taxi_process(i, (i + 1) * 2, i * DEPARTURE_INTERVAL)
    for i in range(NUM_TAXIS)
}
sim = Simulator(taxis)
sim.run(180)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 1      Event(time=5, proc=1, action='leave garage')
taxi: 0  Event(time=6, proc=0, action='pick up passenger')
taxi: 1      Event(time=6, proc=1, action='pick up passenger')
taxi: 2          Event(time=10, proc=2, action='leave garage')
taxi: 1      Event(time=12, proc=1, action='drop off passenger')
taxi: 0  Event(time=16, proc=0, action='drop off passenger')
taxi: 2          Event(time=16, proc=2, action='pick up passenger')
taxi: 2          Event(time=17, proc=2, action='drop off passenger')
taxi: 1      Event(time=19, proc=1, action='pick up passenger')
taxi: 2          Event(time=20, proc=2, action='pick up passenger')
taxi: 0  Event(time=22, proc=0, action='pick up passenger')
taxi: 2          Event(time=22, proc=2, action='drop off passenger')
taxi: 1      Event(time=23, proc=1, action='drop off passenger')
taxi: 0  Event(time=28, proc=0, action='drop off passenger')
taxi: 1      Event(time=31, proc=1, action='pick up p